In [1]:
from  __future__ import print_function
import torch 

In [18]:
x = torch.empty(5,3)
y = torch.rand(5,3)
print(x)
print(y)
print(torch.tanh(x+y))

tensor([[-1.9393e+00,  4.5562e-41, -1.9393e+00],
        [ 4.5562e-41,  1.6255e-43,  1.5554e-43],
        [ 1.5975e-43,  1.3873e-43,  1.4574e-43],
        [ 6.4460e-44,  1.4153e-43,  1.5274e-43],
        [ 1.5695e-43,  1.6255e-43,  1.6956e-43]])
tensor([[ 0.3226,  0.7654,  0.9202],
        [ 0.9469,  0.8257,  0.2831],
        [ 0.5871,  0.4555,  0.7124],
        [ 0.2747,  0.6945,  0.7272],
        [ 0.1420,  0.0884,  0.1037]])
tensor([[-0.9242,  0.6442, -0.7695],
        [ 0.7384,  0.6782,  0.2758],
        [ 0.5278,  0.4264,  0.6122],
        [ 0.2680,  0.6009,  0.6213],
        [ 0.1411,  0.0882,  0.1033]])


In [26]:
x.view(-1)

tensor([-1.9393e+00,  4.5562e-41, -1.9393e+00,  4.5562e-41,  1.6255e-43,
         1.5554e-43,  1.5975e-43,  1.3873e-43,  1.4574e-43,  6.4460e-44,
         1.4153e-43,  1.5274e-43,  1.5695e-43,  1.6255e-43,  1.6956e-43])

In [33]:
x = torch.ones(2,2, requires_grad=True)
print(x)
y = x**x
print(y)
print(y.grad_fn)
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[ 1.,  1.],
        [ 1.,  1.]])
tensor([[ 1.,  1.],
        [ 1.,  1.]])
tensor([[ 3.,  3.],
        [ 3.,  3.]]) tensor(3.)


#### .requires_grad_( ... ) changes an existing Tensor’s requires_grad flag in-place. The input flag defaults to True if not given.

In [34]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradients
#### Let’s backprop now Because out contains a single scalar, out.backward() is equivalent to out.backward(torch.tensor(1)).

In [36]:
out.backward()

In [38]:
print(x.grad)

tensor([[ 1.5000,  1.5000],
        [ 1.5000,  1.5000]])


#### You can also stops autograd from tracking history on Tensors with requires_grad=True by wrapping the code block in with torch.no_grad():

In [39]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


## Neural Networks
#### Neural networks can be constructed using the torch.nn package.

> #### A typical training procedure for a neural network is as follows:
> * Define the neural network that has some learnable parameters (or weights)
> * Iterate over a dataset of inputs
> * Process input through the network
> * Compute the loss (how far is the output from being correct)
> * Propagate gradients back into the network’s parameters
> * Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

## Define the network
### Let’s define this network:

In [43]:
import torch.nn as nn 
import torch.nn.functional as F 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels
        # 5X5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation : y = Wx +b 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
                
        # Max pool over 2X2 window 
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        
        # can specify single number if size is square 
        x = F.max_pool2d(F.relu(self.conv2(x)), (2))
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # All dimension except batch dimension
        size = x.size()[1:] 
        num_features = 1 
        for s in size:
            num_features *=s
        return num_features
    
    
       
net = Net()
print(net)
            
        
    

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
